# Load and execute the model

In [1]:
import platform
print("python version " + platform.python_version())
import sys
import json
import re
import os
from os.path import exists
if exists("config.cfg"):
    from configparser import ConfigParser
    config = ConfigParser()
    config.read("config.cfg")
    paths = config.get("script", "syspaths").split(";")
    for path in paths:
        sys.path.append(path)
        
graphs = [
    {
        'trial':'B4',
        'content': 'OD',
        'experimental_data': True
    }
]

python version 3.7.6


In [ ]:
from datetime import datetime
%run mscommfitting.py
mscommfit = MSCommFitting()
mscommfit.load_data(phenotypes_csv_path = 'PhenotypeStoichiometry.csv', 
                           signal_tsv_paths = {'RFP_pf-ec_4-29-22.txt':'ecoli', 'GFP_pf-ec_4-29-22.txt':'pf', 'OD_pf-ec_4-29-22.txt':'OD'},
                           media_conc_path = 'media_conc.csv',
                           species_abundance_path = 'species_abundances.csv',
                           carbon_conc_series = {
                               'cpd00136': {'B':0, 'C': 0, 'D': 1, 'E': 1, 'F': 4, 'G': 4},
                               'cpd00179': {'B':5, 'C': 5, 'D':5, 'E': 5, 'F': 5, 'G': 5},
                               },
                           ignore_trials = {'rows': ['C', 'D', 'E', 'F']}, # 'wells': ['C2', 'G3', 'C12']},  # 'columns': [6,7,8,9,10,11,12]},
                           # ignore_timesteps = [x for x in range(20,167)],
                           zip_path='mscomfit.zip'
                   )
print(datetime.today())
mscommfit.define_problem(parameters={
    'v':0.4,
    'cvmin': 100,
    'bcv': 100,
    'diffpos': 100,
    'diffneg': 100
    }, zip_name='simple_full_community.zip',flexible_consumption = True,metabolites_to_track = ["cpd00179","cpd00029","cpd00136"])
print(datetime.today())
%time mscommfit.compute(graphs, zip_name='simple_full_community.zip')
print('growth:', mscommfit.parameters['v'], 'objective value:', mscommfit.problem.objective.value, '\n\n\n\n')
print(datetime.today())

# Change the parameters

In [ ]:
mscommfit.change_parameters(vmax=-.4, zip_name='simple_full_community_edited.zip')  # The Vmax argument can be either a number or a dictionary that is organized by ["time"]["trial"], just as the naming scheme of the variables and constraints
%time mscommfit.compute(graphs, 'simple_full_community_edited.zip')
print('growth:', mscommfit.parameters['v'], 'objective value:', mscommfit.problem.objective.value, '\n\n\n\n')
print(datetime.today())

Done loading the JSON: 0.04457041666666726 min


/Users/chenry/opt/anaconda3/lib/python3.7/zipfile.py:1506 UserWarning: Duplicate name: 'mscommfitting.json'


Done exporting the model: 0.4064566499999993 min
